In [3]:
from splinter import Browser
import re
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
import pandas as pd
import numpy as np
import re
from pathlib import Path  

In [4]:
#Set up Splinter 
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless = False)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/terencelin/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


In [45]:
nvidia_gpus = ['https://www.zoolert.com/computers/videocards/nvidia/rtx-3070/instockhistory.php',
               'https://www.zoolert.com/computers/videocards/nvidia/rtx-3070-ti/instockhistory.php',
              'https://www.zoolert.com/computers/videocards/nvidia/rtx-3080-ti/instockhistory.php',
               'https://www.zoolert.com/computers/videocards/nvidia/rtx-3090/instockhistory.php',
               'https://www.zoolert.com/computers/videocards/nvidia/rtx-3080/instockhistory.php',
               'https://www.zoolert.com/computers/videocards/nvidia/rtx-3060ti/instockhistory.php',
               'https://www.zoolert.com/computers/videocards/nvidia/rtx-3060/instockhistory.php'
              ]
models = ['rtx-3070','rtx-3070ti','rtx-3080ti','rtx-3090','rtx-3080','rtx-3060ti','rtx-3060']
chipset = []
descriptions = []
time = []
stores = []
for values,references in enumerate(nvidia_gpus):
    browser.visit(references)
    html = browser.html
    gpu_soup = soup(html,'html.parser')
    browser.find_by_name('history_length').first.select('-1')

for values,references in enumerate(nvidia_gpus):
    browser.visit(references)
    html = browser.html
    gpu_soup = soup(html,'html.parser')
    data = gpu_soup.find_all('tr',role = "row")

    for value,link in enumerate(data):
        descriptions.append(link.get_text(separator = "\n"))
        if( 'T' not in descriptions[value].strip().split("\n")[-1] ):  
            time.append(descriptions[value].strip().split("\n")[-1])
            chipset.append(models[values])
            stores.append(descriptions[value].strip().split("\n")[0])
print(len(time))
company = ['Nvidia'] * len(chipset)

1767


In [46]:
amd_gpus = ['https://www.zoolert.com/computers/videocards/radeon/rx5700/instockhistory.php',
           'https://www.zoolert.com/computers/videocards/radeon/6600xt/instockhistory.php',
           'https://www.zoolert.com/computers/videocards/radeon/rx6600/instockhistory.php',
           'https://www.zoolert.com/computers/videocards/radeon/rx6700xt/instockhistory.php',
           'https://www.zoolert.com/computers/videocards/radeon/rx6800xt/instockhistory.php',
           'https://www.zoolert.com/computers/videocards/radeon/rx6900xt/instockhistory.php'
           ]

models = ['rx5700','600xt','rx6600','rx6700xt','rx6800xt','rx6900xt']
for values,references in enumerate(amd_gpus):
    browser.visit(references)
    html = browser.html
    gpu_soup = soup(html,'html.parser')
    browser.find_by_name('history_length').first.select('-1')

for values,references in enumerate(amd_gpus):
    browser.visit(references)
    html = browser.html
    gpu_soup = soup(html,'html.parser')
    data = gpu_soup.find_all('tr',role = "row")

    for value,link in enumerate(data):
        descriptions.append(link.get_text(separator = "\n"))
        if( 'T' not in descriptions[value].strip().split("\n")[-1] ):  
            time.append(descriptions[value].strip().split("\n")[-1])
            chipset.append(models[values])
            stores.append(descriptions[value].strip().split("\n")[0])
            company.append("AMD")

In [47]:
final_conversion = []
time = [value for value in time if value != "Total Time"]
for timestamps, stamps in enumerate(time):
        
    total_time = 0
    cleaned_time = [int(ele) for ele in re.split('[a-z]',stamps) if ele.strip()]
    if len(cleaned_time) is 3: 
        scalar =1440
        total_time = scalar * cleaned_time[0]
        scalar = 60
        total_time = total_time +(scalar*cleaned_time[1])
        scalar = 1
        total_time = total_time + (scalar*cleaned_time[2])
    elif len(cleaned_time) is 2: 
        scalar = 60
        total_time = total_time +(scalar*cleaned_time[0])
        scalar = 1
        total_time = total_time + (scalar*cleaned_time[1])
    else:
        total_time = cleaned_time[0]
    final_conversion.append(total_time)


<>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
/var/folders/ww/ps0ny6hn32l60w_94q9rtv3w0000gn/T/ipykernel_24186/2829807125.py:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if len(cleaned_time) is 3:
/var/folders/ww/ps0ny6hn32l60w_94q9rtv3w0000gn/T/ipykernel_24186/2829807125.py:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif len(cleaned_time) is 2:


In [48]:
print(len(final_conversion))

3982


In [49]:
print(len(stores),'',len(company),'',len(final_conversion),len(descriptions))

3982  3982  3982 4007


In [51]:
df= pd.DataFrame({
    'stores':stores,
    'Company':company,
    'chipset':chipset,
    'time':final_conversion})
df

,stores,Company,chipset,time
0,Amazon,Nvidia,rtx-3070,712
1,Amazon,Nvidia,rtx-3070,40
2,Amazon,Nvidia,rtx-3070,4
3,Amazon,Nvidia,rtx-3070,548
4,Amazon,Nvidia,rtx-3070,500
...,...,...,...,...
3977,Amazon,AMD,rx6900xt,160
3978,Amazon,AMD,rx6900xt,3475
3979,Amazon,AMD,rx6900xt,7
3980,Amazon,AMD,rx6900xt,4


In [52]:
df.Company.unique()

array(['Nvidia', 'AMD'], dtype=object)

In [53]:
df.stores.unique()

array(['Amazon', 'Amazon CA', 'SamsClub', 'BestBuy'], dtype=object)

In [54]:
filepath = Path('CSV/stock_and_time.csv')
df.to_csv(filepath)